<a href="https://colab.research.google.com/github/Untick/InspectrumClinic_HR_gr1/blob/main/Kiparenko%20Nikolai/INSPECTRUM_CLINIC_HR_RECSYS_GR1_AUTO_ML_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INSPECTRUM CLINIC HR RECSYS

Загружаем библиотеки

In [1]:
# Загрузка датасетов из облака
import gdown

# Функции операционной системы
import os

# Работа с массивами данных
import numpy as np

# Работа с таблицами
import pandas as pd

# автоматически парсит строку и сохранит ее в переменной datetime
import datetime

# Токенизатор для преобразование текстов в последовательности
from tensorflow.keras.preprocessing.text import Tokenizer

# Отрисовка графиков
import matplotlib.pyplot as plt
import seaborn as sns # Будем визулизировать данные этой библиотекой

# Метрики для расчета ошибок
from sklearn.metrics import mean_squared_error, mean_absolute_error


Запускаем H2O

In [6]:
# H2OAutoML
!pip install h2o
import h2o
from h2o.automl import H2OAutoML
h2o.init()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 MB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for h2o: filename=h2o-3.42.0.1-py2.py3-none-any.whl size=249152215 sha256=148da09e3083b79b8b4418a49272714d3c93aa20becd91f6b5b037c52ea80302
  Stored in directory: /root/.cache/pip/wheels/2e/2f/04/ef0554c89bcb5d38a6fc082a87f497933109faefd94dcdc3d2
Successfully built h2o
Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.19" 2023-04-18; OpenJDK Runtime Environment (build 11.0.19+7-post-Ubuntu-0ubuntu120.04.1); OpenJDK 64-Bit Server VM (build 11.0.19+7-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.10/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpq1k92u19
  JVM stdout: /tmp/tmpq1k92u19/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpq1k92u19/h2o_unknownUser_star

H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.1
H2O_cluster_version_age:,"28 days, 19 hours and 17 minutes"
H2O_cluster_name:,H2O_from_python_unknownUser_4ybfbc
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.170 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Грузим файл который нужно проанализировать

In [7]:
#Грузим файл который нужно проверить
path = "https://docs.google.com/spreadsheets/d/1tcfBgGY98cFx1DIvmoRy7goHXNcnaBZ7/export?format=xlsx"
gdown.download(path,'dataS.xlsx', quiet=True)

/usr/local/lib/python3.10/dist-packages/gdown/parse_url.py:35: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=None
  warnings.warn(


'dataS.xlsx'

In [8]:
#Читаем файл
data = pd.read_excel (r'dataS.xlsx', index_col=0)
data.to_csv('data.csv')
data.head(5)
df = h2o.import_file("data.csv",header =1)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


Загружаем предварительно обученную модель

In [2]:
gdown.download("https://s228vla.storage.yandex.net/rdisk/a0049622545167d42dc0574914962413581c2a39617f8167bc18e88198b841f5/64b81497/2V2cgGRqetkaGS3kvhKGJ2tQunQ9Me4bWPoIN8VGagSTlr-KeAiVNYF57MM1SGcoKyQ4ENEUsw_FjqfcZ_8E-Q==?uid=0&filename=StackedEnsemble_AllModels_1_AutoML_4_20230712_144139.zip&disposition=attachment&hash=pU%2BZDc/ZiaiwXeiE4NoBFMxa5GiP%2BVQEPafdQMPEo9OhqCkg8QZjYyS2Db%2BAxfKnq/J6bpmRyOJonT3VoXnDag%3D%3D&limit=0&content_type=application%2Fzip&owner_uid=269742668&fsize=649647&hid=e5db833f3b9c98610b8fbdeb16b32bc8&media_type=compressed&tknv=v2&rtoken=a7Gj6Kie8Jkj&force_default=no&ycrid=na-bdba2a4b04212699e9a6a845fbcb7e23-downloader21h&ts=600d9d82d13c0&s=c787fb38893feb9b766806ec3ee5c7df79ce3b00ac9d77e7f4b1d21172981446&pb=U2FsdGVkX18sZqUx0KntHm7yRbtQmUA5f2qrxDKKqH_cBwKqswSXG_xX6cPXPLunyEzvbuqvqWUBX6wsdZsSoBAsPoMfW36SvZk3oZqOzME",'model.zip', quiet=True)

'model.zip'

In [9]:
imported_model = h2o.import_mojo("model.zip")

generic Model Build progress: |██████████████████████████████████████████████████| (done) 100%


Получаем результат

In [10]:
preds = imported_model.predict(df)

preds = preds.as_data_frame()
df = df.as_data_frame()
df = data.join(preds)
df.to_csv('Ответы.csv')
df.head(5)

generic prediction progress: |███████████████████████████████████████████████████| (done) 100%


/usr/local/lib/python3.10/dist-packages/h2o/job.py:81: UserWarning: Test/Validation dataset column 'Пол, возраст' has levels not trained on: ["", "5 раза снизилась стоимость привлечения лида", "B2G.", "MVC ASP.NET", "P&L - моделью.", "PEST анализ", "ReactJS", "Redmine
Scrum
Управление продуктом
Jira
Kanban
MVP
Agile
Waterfall
UX/UI
CJM
Подготовка ТЗ
User Stories
CustDev
Unit-экономика
Спринты
Водопад
Product менеджер", "SEO продвижения сайтов", "SMM.", ...378 not listed..., "таргетированной рекламы Вконакте", "тестировщиками.", "технические специалисты);", "технологии.", "товарных категорий и продуктов", "управление маркетингом", "холодные звонки", "что повлекло снижение затрат на привлечение тех", "что показалось интереснее. Один раз живём :)"", "что часть продаж через B2B2C (т.е. через дизайнеров интерьера). Их потребности пересеклись с потребностями остальных покупателей. CPS (стоимость привлеченного клиента) снизилась до 3.5 тыс. руб."]
  warnings.warn(w)
/usr/local/lib/python3.10/

,Дата отклика,"Пол, возраст",ЗП,Ищет работу на должность,Город,Занятость,График,Опыт (),Последнее/нынешное место работы,Последняя/нынешняя должность,Выполняемые задачи на прошлых местах работ,Обо мне,Ключевые навыки,Образование и ВУЗ,Обновление резюме,Подходит - да/нет,predict,да,нет
0,2023-02-23 00:00:00,Женщина,90 000 руб.,Growth Product Manager,"Индонезия, Не готова к переезду, не готова к к...","проектная работа, частичная занятость, полная ...",удаленная работа,Опыт работы —5 лет 10 месяцев,PT MYBALI APP GROUP,Менеджер по продукту,Маркетинговые исследования\nИсследование рынка...,NaN,Анализ рынка\nУправление проектами\nProduct Ma...,"Высшее, 2023, Нетоло́гия, 2014, Университет Ро...",23 февраля 2023 в 16:30,да,да,0.999591,0.000409
1,2023-01-11 00:00:00,Женщина,не указана,Growth Product Manager,"Москва, Готова к переезду, готова к командировкам",полная занятость,"удаленная работа, гибкий график, полный день",Опыт работы —11 лет 11 месяцев,Банк Открытие,Product owner трайб сбережения и инвестиции,Участие в команде роста\nВ2В\n\nЗадачи:\n- Раз...,NaN,Самомотивация\nТаймменеджмент\nКреативность\nЧ...,"Высшее, 2022, GeekBrains, 2010, Московский худ...",8 февраля 2023 в 09:15,да,да,0.999341,0.000659
2,2023-02-27 00:00:00,"Мужчина, 33 года, родился 23 августа 1989",150 000 руб.,Growth Product Manager,"Санкт-Петербург, Готов к переезду: Москва, гот...","проектная работа, частичная занятость, полная ...","удаленная работа, гибкий график, полный день",Опыт работы —7 лет 2 месяца,DIKIY LLC,,Запуск бренда и производства игристого кваса D...,"Чем бы я ни занимался с 2014 года, когда решил...",Продвижение бренда\nПланирование маркетинговых...,"Высшее, 2013, Санкт-Петербургский государствен...",28 февраля 2023 в 09:41,да,да,0.999253,0.000747
3,2023-01-11 00:00:00,"Мужчина, 32 года, родился 7 декабря 1990",не указана,Growth Product Manager,"Москва, м. Молодежная, Хочу переехать, готов к...","стажировка, частичная занятость, полная занятость","удаленная работа, гибкий график, полный день",Опыт работы —8 лет 10 месяцев,WeInvest,Product Manager (team leader),1. Исследование рынка\n2. Определение целевой ...,"Я проактивный, коммуникабельный, позитивный, д...",Управление проектами\nОрганизаторские навыки\n...,"Магистр, 2014, Московский государственный техн...",24 января 2023 в 12:07,да,нет,0.002273,0.997727
4,2023-01-11 00:00:00,"Мужчина, 26 лет, родился 21 февраля 1996",150 000 руб.,Growth Product Manager,"Пермь, Хочу переехать, готов к командировкам","проектная работа, частичная занятость, полная ...","удаленная работа, гибкий график, полный день",Опыт работы —5 лет 9 месяцев,Индивидуальное предпринимательство / частная п...,Product/Project manager,Переговоры со стейкхолдером\n- Составление ТЗ...,Набрал кучу опыта и хочу с ним найти классную ...,Управление персоналом\nЗаключение договоров\nА...,"Высшее, 2021, Российская академия народного хо...",6 февраля 2023 в 11:18,да,нет,0.004218,0.995782
